In [58]:
# !pip install crewai crewai-tools langchain ollama

In [59]:
# !pip install langchain_openai

In [60]:
from crewai import Agent, LLM, Crew, Process, Task, TaskOutput, CrewOutput
from crewai.tools import tool
from langchain_openai import ChatOpenAI

In [61]:
import warnings
warnings.filterwarnings('ignore')

# LLM

In [62]:
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI # Import ChatOpenAI

# Retrieve the API key from Colab's user data
key = userdata.get('4.1')

# Set the OPENAI_API_KEY environment variable
# os.environ["OPENAI_API_KEY"] = key

# Initialize the ChatOpenAI model
openai_llm = LLM(
    api_key=key,
    base_url="https://models.github.ai/inference",
    model="openai/gpt-4.1",
)

In [63]:
# openai_llm.invoke("hello")

In [64]:
import torch
from diffusers import StableDiffusionXLPipeline
# from IPython.display import display # Remove display
from crewai.tools import tool
from PIL import Image # Import Image
import os
import time # Import time for timestamp in filename

# Initialize the pipeline once
pipe = StableDiffusionXLPipeline.from_pretrained(
    "segmind/SSD-1B",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
).to("cuda")

@tool("generate_image_with_segmind")
def generate_image_with_segmind(prompt: str) -> str: # Change return type hint to str
    """
    Generates an image with Segmind (SSD-1B model) based on the given prompt using diffusers,
    saves the image to a local file, and returns the file path.
    """
    print(f"Generating image with prompt: {prompt}") # Debugging print

    image = pipe(
        prompt=prompt,
        negative_prompt="ugly, darkness, blurry, poor quality",
        width=1500 - (1500 % 8),
        height=750 - (750 % 8),
        guidance_scale=7,
        num_inference_steps=28
    ).images[0]

    # Save the image to a local file
    timestamp = int(time.time())
    filename = f"generated_image_{timestamp}.png"
    image_path = os.path.join(".", filename) # Save in the current directory
    image.save(image_path)
    print(f"Image saved to: {image_path}") # Debugging print

    # display(image) # Remove display

    return image_path # Return the image file path as a string

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

# Research Agent

In [65]:
research_agent = Agent(
			role="Researcher",
			goal="Conduct in-depth research and gather information about {topic}.",
			backstory=
            "You're working on planning a blog article "
            "about the topic: {topic}."
            "You collect information that helps the "
            "audience learn something "
            "and make informed decisions. "
            "Your work is the basis for "
            "the Content Writer to write an article on this topic.",
			allow_delegation=False,
			llm=openai_llm,
			verbose=True
		)

# Writer Agent

In [66]:
writer_agent = Agent(
            role="Writer",
            goal="Write compelling Instagram captions (short-form + long-form version) based on research about {topic}, including relevant hashtags and a call to action.",
            backstory=
            "You are a creative content writer specializing in "
            "compelling Instagram captions. "
            "You take the research provided by the Research Agent "
            "and craft engaging short-form and long-form captions "
            "optimized for Instagram, including relevant hashtags "
            "and a call to action. You work closely with the Reviewer Agent "
            "to ensure the captions are polished and effective.",
            allow_delegation=False,
            llm=openai_llm,
            verbose=True,
            memory=True
        )

# Reviewer Agent

In [67]:
reviewer_Agent = Agent(
            role="Reviewer",
            goal="Review and refine Instagram captions for {topic}.",
            backstory=
            "You are an editor who receives Instagram captions "
            "from the Content Writer. "
            "Your goal is to review the captions to ensure they are "
            "polished, grammatically correct, and optimized for clarity, "
            "engagement, and tone. You also verify that they align with the "
            "research and project objectives.",
            allow_delegation=False,
            llm=openai_llm,
            verbose=True,
            memory=True
        )

# Image Generator Agent

In [68]:
image_agent = Agent(
            role="Designer",
            goal="Generate 2–3 strong text-to-image prompts for {topic}, "
            "and always call the generate_image_with_segmind tool with at least one prompt "
            "to produce and display a real image using diffusers.",
            backstory=
            "You specialize in creating image prompts and generating visuals "
            "with an image generation tool using diffusers. Your job is to ensure the images align with the "
            "Instagram caption content and are displayed in the notebook."
            ,
            allow_delegation=False,
            llm=openai_llm,
            verbose=True,
            memory=True,
            tools=[generate_image_with_segmind]
        )

# Define Tasks

In [69]:
research_task = Task(
    description=(
        "Conduct in-depth research on the topic: {topic}."
        "Identify key points, interesting facts, and potential angles "
        "that would be engaging for an Instagram audience."
        "Compile your findings into a concise summary."
    ),
    expected_output='A concise summary of key points and interesting facts about {topic} for an Instagram audience.',
    agent=research_agent,
)

writer_task = Task(
    description=(
        "Using the research summary provided, write two versions of an Instagram caption for {topic}: "
        "1. A short-form caption (under 150 characters including hashtags) "
        "2. A long-form caption (up to 2200 characters including hashtags) "
        "Both captions should be compelling, include a relevant call to action, and suggest relevant hashtags."
    ),
    expected_output='Two Instagram captions (short-form and long-form) for {topic}, including a call to action and relevant hashtags.',
    agent=writer_agent,
)

reviewer_task = Task(
    description=(
        "Review the short-form and long-form Instagram captions for {topic} provided by the writer."
        "Ensure they are grammatically correct, clear, engaging, and maintain an appropriate tone."
        "Refine the captions for maximum impact and verify the relevance of the suggested hashtags."
        "Provide the final, polished versions of both captions and the finalized list of hashtags."
    ),
    expected_output='The final, polished short-form and long-form Instagram captions for {topic} and a finalized list of relevant hashtags.',
    agent=reviewer_Agent,
)

image_prompt_task = Task(
    description=(
        "Based on the topic {topic} and the content of the Instagram captions "
        "(both short and long form), generate 2 to 3 detailed and creative "
        "text-to-image prompts. For EACH generated prompt, use "
        "the generate_image_with_segmind tool to produce and display a real image."
    ),
    expected_output=(
        "2 to 3 detailed text-to-image prompts for {topic} based on the Instagram caption content, "
        "and the file paths to 2 to 3 generated images that were saved locally using generate_image_with_segmind."
    ),
    agent=image_agent,
)

# Create Crew

In [70]:
content_crew = Crew(
    agents=[research_agent, writer_agent, reviewer_Agent, image_agent],
    tasks=[research_task, writer_task, reviewer_task, image_prompt_task],
    process=Process.sequential,
    verbose=True,
)

In [71]:
# openai_llm.invoke("hello")

# Execute Crew

In [72]:
from IPython.display import Markdown
# Define the topic
topic = "The Future of AI in Healthcare"

# Kickoff the crew process
result = content_crew.kickoff(inputs={'topic': topic})
# Print the results
print("## Here is the Instagram Content Package ##")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0ab19c06-4ae4-44c5-86c2-3287d05674ca                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Task: Conduct in-depth research on the topic: The Future of AI in Healthcare.Identify key points, interesting  │
│  facts, and potential angles that would be engaging for an Instagram audience.Compile your findings into a      │
│  concise summary.                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The Future of AI in Healthcare: Key Points & Interesting Facts for Instagram                                   │
│                                                                                                                 │
│  1. Personalized Medicine Gets Smarter                                                                          │
│  AI can analyze huge amounts of genetic, lifestyle, and medical data—helping doctors tailor treatments to each  │
│  individual. Think: custom cancer therapies and precise drug recommendations!                                   │
│                                                                                                                 │
│  2. Faster, More Accurate Diagnoses                                                                             │
│  AI-powered tools (like Google’s DeepMind, IBM Watson Health) are already helping radiologists spot diseases    │
│  in medical images, sometimes more accurately than human experts. Early detection of cancer, heart disease,     │
│  and rare illnesses is becoming a reality.                                                                      │
│                                                                                                                 │
│  3. Virtual Health Assistants                                                                                   │
│  Smart chatbots and voice assistants are supporting patients 24/7—answering questions, reminding you to take    │
│  medicine, and booking appointments, making healthcare more accessible (especially in rural or underserved      │
│  areas).                                                                                                        │
│                                                                                                                 │
│  4. Safer Surgeries                                                                                             │
│  Robotic systems powered by AI (think Da Vinci Surgical System) can assist surgeons, reducing errors, speeding  │
│  up recovery, and allowing for minimally invasive procedures.                                                   │
│                                                                                                                 │
│  5. Wearable Devices & Remote Monitoring                                                                        │
│  From smartwatches to patches, AI analyzes real-time data to spot abnormalities—like irregular heartbeats or    │
│  blood sugar spikes—alerting you (and your doctor) instantly.                                                   │
│                                                                                                                 │
│  6. Fighting Disease Outbreaks                                                                                  │
│  AI can predict and track disease outbreaks (like COVID-19) by scanning global data, helping public health      │
│  experts respond faster and prevent further spread.                                                             │
│                                                                                                                 │
│  7. Tackling Administrative Hassles                                                                             │
│  Automated systems powered by AI are reducing paperwork

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7ae3f623-c608-4ba9-b456-f0241029e1a1                                                                     │
│  Agent: Researcher                                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Designer                                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here are 3 detailed text-to-image prompts for The Future of AI in Healthcare, based on your Instagram          │
│  captions—along with the corresponding file paths to the generated images:                                      │
│                                                                                                                 │
│  Prompt 1:                                                                                                      │
│  A modern hospital scene where a doctor consults with an AI-powered holographic assistant, surrounded by data   │
│  visualizations, patient charts, and glowing genetic codes on digital screens, symbolizing personalized         │
│  medicine and collaboration, sleek futuristic setting, vibrant lighting, diverse healthcare team.               │
│  Image Path: ./generated_image_1759242994.png                                                                   │
│                                                                                                                 │
│  Prompt 2:                                                                                                      │
│  A radiology lab with a radiologist examining medical images on large monitors, as an AI system overlays        │
│  highlighted areas and predictive analytics, showing virtual assistants and real-time disease detection, clean  │
│  futuristic clinic, digital displays glowing, a sense of high-tech precision.                                   │
│  Image Path: ./generated_image_1759243023.png                                                                   │
│                                                                                                                 │
│  Prompt 3:                                                                                                      │
│  A patient at home wearing a sleek AI-powered smartwatch and health monitoring patch, surrounded by floating    │
│  data graphs and notifications displayed in the air, real-time analysis alerts appearing on the device, cozy    │
│  high-tech living room setting, vibrant colors, focus on wellness and connectivity.                             │
│  Image Path: ./generated_image_1759243047.png                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6110cd36-7690-40ec-b680-2797548edeed                                                                     │
│  Agent: Designer                                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0ab19c06-4ae4-44c5-86c2-3287d05674ca                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Here are 3 detailed text-to-image prompts for The Future of AI in Healthcare, based on your      │
│  Instagram captions—along with the corresponding file paths to the generated images:                            │
│                                                                                                                 │
│  Prompt 1:                                                                                                      │
│  A modern hospital scene where a doctor consults with an AI-powered holographic assistant, surrounded by data   │
│  visualizations, patient charts, and glowing genetic codes on digital screens, symbolizing personalized         │
│  medicine and collaboration, sleek futuristic setting, vibrant lighting, diverse healthcare team.               │
│  Image Path: ./generated_image_1759242994.png                                                                   │
│                                                                                                                 │
│  Prompt 2:                                                                                                      │
│  A radiology lab with a radiologist examining medical images on large monitors, as an AI system overlays        │
│  highlighted areas and predictive analytics, showing virtual assistants and real-time disease detection, clean  │
│  futuristic clinic, digital displays glowing, a sense of high-tech precision.                                   │
│  Image Path: ./generated_image_1759243023.png                                                                   │
│                                                                                                                 │
│  Prompt 3:                                                                                                      │
│  A patient at home wearing a sleek AI-powered smartwatch and health monitoring patch, surrounded by floating    │
│  data graphs and notifications displayed in the air, real-time analysis alerts appearing on the device, cozy    │
│  high-tech living room setting, vibrant colors, focus on wellness and connectivity.                             │
│  Image Path: ./generated_image_1759243047.png                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): ## Here is the Instagram Content Package ##
Here are 3 detailed text-to-image prompts for The Future of AI in Healthcare, based on your Instagram captions—along with the corresponding file paths to the generated images:

Prompt 1:
A modern hospital scene where a doctor consults with an AI-powered holographic assistant, surrounded by data visualizations, patient charts, and glowing genetic codes on digital screens, symbolizing personalized medicine and collaboration, sleek futuristic setting, vibrant lighting, diverse healthcare team.
Image Path: ./generated_image_1759242994.png

Prompt 2:
A radiology lab with a radiologist examining medical images on large monitors, as an AI system overlays highlighted areas and predictive analytics, showing virtual assistants and real-time disease detection, clean futuristic clinic, digital displays glowing, a sense of high-tech precision.
Image Path: ./generated_image_1759243023.png



In [73]:
# display(Markdown(str(result))) # Convert result to string

In [74]:
# task_output: TaskOutput = result.tasks_output[0]
# crew_output: CrewOutput = result.output

In [75]:
# Inspect the raw output of the reviewer task to understand its structure
print(result.tasks_output[2].raw)

Short-form Instagram Caption (Polished):
Meet healthcare’s new dream team: AI + doctors! Personalized care, quicker diagnoses, and smarter solutions—your health, reimagined. #FutureofAI #HealthcareInnovation #MedicalTech #WellnessRevolution

Long-form Instagram Caption (Polished):
Imagine receiving medical care tailored just for you, powered by intelligent algorithms that analyze your unique genetic makeup, lifestyle, and health history. 🤖✨ The future of AI in healthcare has arrived—and it’s changing everything!

AI is enabling doctors to create customized treatment plans, from personalized cancer therapies to the most precise drug recommendations. AI-powered tools empower radiologists to spot diseases faster and often more accurately than ever, while virtual assistants are answering questions, reminding you to take medication, and handling bookings—making healthcare truly accessible, anytime and anywhere.

Surgeries are getting safer with robotics like the Da Vinci Surgical System, re

In [77]:
from IPython.display import display, Markdown
from PIL import Image # Import Image (although not strictly needed for printing text)

# Access the raw string output using the 'raw' attribute
reviewer_output_string = result.tasks_output[2].raw  # Assuming reviewer_task is the 3rd task (index 2)
image_task_output = result.tasks_output[3] # Access the full task output for the image task (index 3)
image_output_string = image_task_output.raw # Get the raw string output

# Initialize variables to hold extracted content
short_caption = "Short-form caption not found."
long_caption = "Long-form caption not found."
hashtags = "Hashtags not found."

# Robustly extract captions and hashtags
long_caption_split = reviewer_output_string.split('Long-form Instagram Caption')
if len(long_caption_split) > 1:
    short_caption_part = long_caption_split[0]
    # Extract short caption - assuming it's everything before "Long-form Instagram Caption"
    short_caption = short_caption_part.replace('Short-form Instagram Caption (Polished):', '').strip() # Updated marker based on reviewer output

    hashtags_split = long_caption_split[1].split('Finalized List of Hashtags:')
    if len(hashtags_split) > 1:
        # Extract long caption - everything between "Long-form Instagram Caption" and "Finalized List of Relevant Hashtags:"
        long_caption = hashtags_split[0].replace('(Polished):', '').strip() # Updated marker
        # Extract hashtags - everything after "Finalized List of Relevant Hashtags:"
        hashtags = hashtags_split[1].strip()
    else:
        # If hashtags section is not found after long caption
        long_caption = long_caption_split[1].replace('(Polished):', '').strip() # Updated marker
        hashtags = "Hashtags section marker not found."
else:
    # If "Long-form Instagram Caption" marker is not found at all
    short_caption = reviewer_output_string.strip()
    long_caption = "Long-form caption marker not found."
    hashtags = "Hashtags marker not found."


# Combine the text outputs into a single string
instagram_package_text = f"""
## Complete Instagram Post Package for "{topic}"

### Instagram Captions

**Short-form Caption:**
{short_caption}

**Long-form Caption:**
{long_caption}

### Relevant Hashtags
{hashtags}

### Generated Images and Prompts

{image_output_string}
"""

# Print the combined text package
print(instagram_package_text)

# Acknowledge that images were generated and saved to local files
print("\n## Generated Images (Saved Locally) ##")
print("Images were generated and saved to local files during the execution of the image generation task.")

# Note: Code to display images from task_output.result was removed as it caused errors and images are displayed by the tool.
# The image_task_output.raw string contains the text output from the image agent, which should include filenames.


## Complete Instagram Post Package for "The Future of AI in Healthcare"

### Instagram Captions

**Short-form Caption:**
Meet healthcare’s new dream team: AI + doctors! Personalized care, quicker diagnoses, and smarter solutions—your health, reimagined. #FutureofAI #HealthcareInnovation #MedicalTech #WellnessRevolution

**Long-form Caption:**
Imagine receiving medical care tailored just for you, powered by intelligent algorithms that analyze your unique genetic makeup, lifestyle, and health history. 🤖✨ The future of AI in healthcare has arrived—and it’s changing everything!

AI is enabling doctors to create customized treatment plans, from personalized cancer therapies to the most precise drug recommendations. AI-powered tools empower radiologists to spot diseases faster and often more accurately than ever, while virtual assistants are answering questions, reminding you to take medication, and handling bookings—making healthcare truly accessible, anytime and anywhere.

Surgeries are g

## Summary:

### Data Analysis Key Findings

*   The `generate_image_with_segmind` function was successfully modified to save generated images to local files named `generated_image_[timestamp].png` and return the file path string.
*   The direct display of images using `display(image)` was removed from the image generation function.
*   The final output generation logic was updated to indicate that images are saved locally rather than displayed.
*   The `expected_output` description for the `image_prompt_task` was updated to reflect that file paths to locally saved images will be part of the output.

### Insights or Next Steps

*   Ensure that the directory where images are saved has appropriate write permissions.
*   Consider adding error handling to the image saving process in case of file writing issues.
